In [11]:
import pandas as pd
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import math


%matplotlib inline

In [2]:
aw_custs = pd.read_csv('AdvWorksCusts.csv')
aw_aveMonth = pd.read_csv('AW_AveMonthSpend.csv')
aw_bike=pd.read_csv('AW_BikeBuyer.csv')

In [3]:
age=[]
for x in aw_custs.BirthDate:
    age.append(1998-int(x[:4]))
aw_custs['Age']=age
aw_custs.head().T

,0,1,2,3,4
CustomerID,11000,11001,11002,11003,11004
Title,NaN,NaN,NaN,NaN,NaN
FirstName,Jon,Eugene,Ruben,Christy,Elizabeth
MiddleName,V,L,NaN,NaN,NaN
LastName,Yang,Huang,Torres,Zhu,Johnson
Suffix,NaN,NaN,NaN,NaN,NaN
AddressLine1,3761 N. 14th St,2243 W St.,5844 Linden Land,1825 Village Pl.,7553 Harness Circle
AddressLine2,NaN,NaN,NaN,NaN,NaN
City,Rockhampton,Seaford,Hobart,North Ryde,Wollongong
StateProvinceName,Queensland,Victoria,Tasmania,New South Wales,New South Wales


In [4]:
aw_custs.drop_duplicates(subset='CustomerID', keep='last', inplace=True)
aw_aveMonth.drop_duplicates(subset='CustomerID', keep='last', inplace=True)
aw_bike.drop_duplicates(subset='CustomerID', keep='last', inplace=True)

In [5]:
features=aw_custs['CountryRegionName']
enc=preprocessing.LabelEncoder()
enc.fit(features)
features=enc.transform(features)
ohe=preprocessing.OneHotEncoder()
encoded=ohe.fit(features.reshape(-1,1))
features = encoded.transform(features.reshape(-1,1)).toarray()
print(features.shape)

(16404, 6)


In [6]:
def encode_string(feature):
    enc=preprocessing.LabelEncoder()
    enc.fit(feature)
    enc_feature=enc.transform(feature)
    ohe=preprocessing.OneHotEncoder()
    encoded=ohe.fit(enc_feature.reshape(-1,1))
    return encoded.transform(enc_feature.reshape(-1,1)).toarray()
columns=['Education',
 'Occupation',
 'Gender',
 'MaritalStatus',]
for col in columns:
    temp=encode_string(aw_custs[col])
    features=np.concatenate([features,temp],axis=1)
print(features.shape)

(16404, 20)


In [7]:
features=np.concatenate([features,np.array(aw_custs[['HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'Age']])],axis=1)
features.shape

(16404, 26)

In [ ]:
aw_aveMonth

In [8]:
nr.seed(123)
labels=np.array(aw_bike['BikeBuyer'])
index=range(features.shape[0])
index=ms.train_test_split(index,test_size=3000)
x_train=features[index[0],:]
y_train=np.ravel(labels[index[0]])
x_test=features[index[1],:]
y_test=np.ravel(labels[index[1]])

In [9]:
scaler = preprocessing.StandardScaler().fit(x_train[:,-6:])
x_train[:,-6:]=scaler.transform(x_train[:,-6:])
x_test[:,-6:]=scaler.transform(x_test[:,-6:])
print(x_train.shape)
x_train[:5,:]

(13404, 26)


array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  6.94797354e-01,
        -4.37017607e-01, -6.56034638e-01,  5.93113487e-01,
         9.44771303e-01,  2.90623161e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00, -1.43926858e+00,
         4.37539575e-01, -6.56034638e-01, -1.50829159e-03,
         7.15

In [ ]:
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(x_train, y_train)

In [ ]:
logistic_mod.score(x_test,y_test)

In [ ]:
logistic_mod.predict_proba(x_test)

In [ ]:
nr.seed(1234)
rf_cla = RandomForestClassifier()
rf_cla.fit(x_train, y_train)

In [ ]:
rf_cla.score(x_test, y_test)

In [ ]:
nr.seed(111)
ab_cla=AdaBoostClassifier()
ab_cla.fit(x_train, y_train)

In [ ]:
ab_cla.score(x_test,y_test)

In [ ]:
nr.seed(222)
nn_cla=MLPClassifier()
nn_cla.fit(x_train,y_train)

In [ ]:
nn_cla.score(x_test,y_test)

In [ ]:
nr.seed(555)
svm_cla=svm.LinearSVC()
svm_cla.fit(x_train, y_train)

In [ ]:
svm_cla.score(x_test,y_test)

In [ ]:
nr.seed(555)
svm_cla=svm.SVC()
svm_cla.fit(x_train, y_train)

In [ ]:
svm_cla.score(x_test,y_test)

In [ ]:
nr.seed(666)
inside=ms.KFold(n_splits=10,shuffle=True)
nr.seed(777)
outside=ms.KFold(n_splits=10,shuffle=True)

In [ ]:
param_grid = {"max_features": [2, 3, 5, 10, 15], "min_samples_leaf":[3, 5, 10, 20]}
nr.seed(888)
rf_cla=RandomForestClassifier(class_weight = "balanced")
nr.seed(999)
rf_cla=ms.GridSearchCV(estimator = rf_cla, param_grid = param_grid, 
                      cv = inside,
                      scoring = 'roc_auc',
                      return_train_score = True)
rf_cla.fit(features,labels)

In [ ]:
nr.seed(22)
rf_mod=RandomForestClassifier(class_weight='balanced',
                             max_features=rf_cla.best_estimator_.max_features,
                             min_samples_leaf=rf_cla.best_estimator_.min_samples_leaf)
rf_mod.fit(x_train, y_train)

In [ ]:
rf_mod.score(x_test,y_test)  

In [12]:
nb_mod =GaussianNB()
nb_mod.fit(x_train,y_train)

GaussianNB(priors=None)

In [13]:
nb_mod.score(x_test,y_test)

0.7753333333333333

In [14]:
nr.seed(77)
inside=ms.KFold(n_splits=10,shuffle=True)
nr.seed(88)
outside=ms.KFold(n_splits=10,shuffle=True)

In [16]:
nr.seed(99)
param_grid={"alpha":[0.0001, 0.001, 0.01, 0.1, 1, 10]}
nb_cla=BernoulliNB()
cla=ms.GridSearchCV(estimator = nb_cla, param_grid = param_grid, 
                      cv = inside,
                      scoring = 'roc_auc',
                      return_train_score = True)
cla.fit(features,labels)
cla.best_estimator_.alpha

10

In [19]:
nb_mod=BernoulliNB(alpha=cla.best_estimator_.alpha)
nb_mod.fit(x_train,y_train)

BernoulliNB(alpha=10, binarize=0.0, class_prior=None, fit_prior=True)

In [20]:
nb_mod.score(x_test,y_test)

0.7676666666666667